In [4]:
import torch.backends.cudnn as cudnn
import torch.nn.init as init
import torch.utils.data as data
import numpy as np
import argparse
import visdom
import pickle

from box_ssd2 import build_ssd_box2
from ssd import build_ssd
from ssd_full_box import build_ssd_full_box 
from data import *
from utils.augmentations import SSDAugmentation
from layers.modules import MultiBoxLoss
import torch.utils.data as data
from torch.utils.data import SubsetRandomSampler
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
from utils import metrics

from box_convolution import BoxConv2d
import torch.nn as nn
%reload_ext autoreload
%autoreload 2

In [5]:
#cfg = voc

#ssd_box_net = build_ssd_box2('train', size=300, num_classes=21)
box_net = build_ssd('train', size=300, num_classes=21)
ssd_full_box = build_ssd_full_box('train', size=300, num_classes=21)

AttributeError: 'BatchNorm2d' object has no attribute 'in_planes'

In [3]:
ssd_full_box

SSD_Full_Box(
  (vgg): ModuleList(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): BoxConv2d()
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): BoxConv2d()
    (6): ReLU(inplace)
    (7): BoxConv2d()
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): BoxConv2d()
    (11): ReLU(inplace)
    (12): BoxConv2d()
    (13): ReLU(inplace)
    (14): BoxConv2d()
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
    (17): BoxConv2d()
    (18): ReLU(inplace)
    (19): BoxConv2d()
    (20): ReLU(inplace)
    (21): BoxConv2d()
    (22): ReLU(inplace)
    (23): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (24): BoxConv2d()
    (25): ReLU(inplace)
    (26): BoxConv2d()
    (27): ReLU(inplace)
    (28): BoxConv2d()
    (29):

In [16]:
box_net

SSD(
  (vgg): ModuleList(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
    (17): Conv2d(256, 

In [15]:
def count_params(model):
    """Count the number of parameters in the current TensorFlow graph """
    param_count = np.sum([np.prod(p.size()) for p in model.parameters()])
    return param_count

#print(count_params(ssd_box_net))
print(count_params(box_net))
print(count_params(ssd_full_box)/count_params(box_net) - 1)
print(count_params(ssd_full_box))

26285486
-0.5591015513276034
11589230


In [6]:
cfg = voc
dataset_train = VOCDetection(root='/home/harryemeric/Project/Code/data/VOCdevkit/',
                               image_sets = [('2012', 'trainval')],
                               transform=SSDAugmentation(cfg['min_dim'],
                                                         MEANS))


frac = 0.01
data_sizes = np.array(len(dataset_train))
data_sizes_sub = (data_sizes*frac).astype(int)
np.random.seed(10)
data_indices = np.random.choice(data_sizes, data_sizes_sub)

data_loader_train = data.DataLoader(dataset_train, 128,
                                  num_workers=4,
                                  sampler=SubsetRandomSampler(data_indices),
                                  shuffle=False, collate_fn=detection_collate,
                                  pin_memory=True)
del dataset_train

batch_iterator = iter(data_loader_train)
for iteration in range(1):
            # load train data
    try:
        images, targets = next(batch_iterator)
    except StopIteration:
        batch_iterator = iter(data_loader_train)
        images, targets = next(batch_iterator)

    if True:
        images = Variable(images.cuda())
        with torch.no_grad():
            targets = [Variable(ann.cuda()) for ann in targets]

In [7]:
ssd_net = build_ssd_box('train', cfg['min_dim'], cfg['num_classes'])
net = ssd_net

net = torch.nn.DataParallel(ssd_net)
cudnn.benchmark = True
net = net.cuda()

net.train()

out = net(images)

RuntimeError: CUDA out of memory. Tried to allocate 632.00 MiB (GPU 0; 11.17 GiB total capacity; 10.40 GiB already allocated; 481.56 MiB free; 9.04 MiB cached)

In [ ]:
help(BoxConv2d)